<h1><b>ShiftKey GenAi Assignemnt<b><h1>

<h5>Submitted by Taskil Mahmud<h5>

In [ ]:
!pip install transformers datasets torch huggingface_hub

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Define the model name and load the tokenizer and model
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Print out which device we're using (GPU or CPU)
print(device)

In [3]:
"""
function to summarize text

@param text: given text
@return: summarized text
"""
def summarize(text):
  inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True).to(device)
  summary_ids = model.generate(inputs["input_ids"], max_length=128, num_beams=4, early_stopping=True)
  return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
#text that is used for testing
sample_text =     """
Person A: Hey, have you heard about the new project management software that our company is planning to implement?

Person B: Yeah, I’ve heard a little bit about it. What’s the latest on that?

Person A: It’s called "TaskFlow." Management is really optimistic about it, believing it will significantly streamline our workflow, particularly with remote teams. The idea is that it will integrate all the tools we currently rely on, like Slack, Trello, and Google Drive, into a single, cohesive platform.

Person B: That does sound intriguing! However, I have some reservations about the learning curve. Do you know if it’s going to be user-friendly?

Person A: From what I’ve seen so far, it looks quite intuitive. They’re also planning to hold several training sessions to ensure everyone gets up to speed quickly. The first session is scheduled for next Monday, so that should help ease any concerns.

Person B: Okay, that definitely puts my mind at ease. I suppose I’ll need to make it a point to attend that session. How does this new tool compare to what we’re currently using?

Person A: It’s reportedly much more efficient. We’ll be able to track project progress with much greater ease and receive real-time updates. Additionally, it features built-in analytics to assist us in monitoring performance and making informed decisions.

Person B: That certainly sounds promising! I just hope it doesn’t launch with a lot of bugs and issues.

Person A: Yeah, that’s always a concern when it comes to new software. However, they’ve been testing it thoroughly for a while now, so here’s hoping that everything goes smoothly at launch.

Person B: Fingers crossed! Let’s hope for the best. Thanks for sharing all this information with me!

Person A: No problem at all! I’m glad to help. I’ll see you at the training session
"""

# Summarize the provided text using the pre-trained model without any fine-tuning.
pre_finetuned_summary = summarize(sample_text)
print("Summary before fine-tuning:", pre_finetuned_summary)


In [ ]:
from datasets import load_dataset

# Load the CNN/DailyMail dataset, which includes articles along with their summaries.
dataset = load_dataset("cnn_dailymail", "3.0.0", split="train")

In [ ]:
# Divide the dataset into training and testing subsets.
dataset_split = dataset.train_test_split(test_size=0.145)

#  Reduce the size of the training set to speed up testing during development.
small_train_dataset = dataset_split['train'].train_test_split(test_size=0.96)['train']
eval_dataset = dataset_split['test']

In [ ]:
"""
@param: examples
@return: preprocessed example
"""
def preprocess_function(examples):
  # Extract the articles from the dataset
  inputs = [doc for doc in examples['article']]

  # Tokenize the articles (inputs), applying padding and truncation to a maximum length of 512.
  model_inputs = tokenizer(inputs, max_length=512, padding="max_length", truncation=True, return_tensors="pt")

  # Tokenize the summaries (labels) using the target tokenizer context
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples['highlights'], max_length=128, padding="max_length", truncation=True, return_tensors="pt")

  # Attach the tokenized summaries as labels to the model inputs
  model_inputs["labels"] = labels["input_ids"]

  # Move the tokenized inputs and labels to the appropriate device (GPU/CPU)
  model_inputs = {k: v.to(device) for k, v in model_inputs.items()}

  return model_inputs

In [ ]:

# Tokenize the small training dataset
tokenized_train_dataset = small_train_dataset.map(preprocess_function, batched=True)

# Tokenize the evaluation dataset
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)

In [ ]:
from transformers import Seq2SeqTrainingArguments

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',              # Directory to save the model checkpoints
    evaluation_strategy="epoch",         # Evaluate the model at the end of every epoch
    learning_rate=2e-5,                  # Learning rate for the optimizer
    per_device_train_batch_size=8,       # Batch size for training
    per_device_eval_batch_size=8,        # Batch size for evaluation
    weight_decay=0.01,                   # Regularization to prevent overfitting
    save_total_limit=3,                  # Only keep the last 3 checkpoints
    num_train_epochs=3,                  # Number of training epochs
    predict_with_generate=True,          # Enable text generation during evaluation
    logging_dir="./logs"                 # Directory for storing training logs
)

In [ ]:
from transformers import Seq2SeqTrainer

# Create the trainer object
trainer = Seq2SeqTrainer(
    model=model,                         # The model to be trained
    args=training_args,                  # The training arguments defined earlier
    train_dataset=tokenized_train_dataset,  # The tokenized training dataset
    eval_dataset=tokenized_eval_dataset,    # The tokenized evaluation dataset
    tokenizer=tokenizer                  # The tokenizer to handle input and output
)

In [ ]:
# Start train
trainer.train()

In [ ]:
# Evaluate the model on the evaluation dataset
metrics = trainer.evaluate()

# Print the evaluation metrics
print(metrics)

In [ ]:

"""
@param: text
@return: summarized text
"""
def summarize(text):
  # Tokenize the input text and place it to the correct device
  inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True).to(device)

  # Generate the summary using the fine-tuned model
  summary_ids = model.generate(inputs["input_ids"], max_length=128, num_beams=4, early_stopping=True)

  # Decode the generated summary back into text and return it
  return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
print(summarize(
    """
Person A: Hey, have you heard about the new project management software that our company is planning to implement?

Person B: Yeah, I’ve heard a little bit about it. What’s the latest on that?

Person A: It’s called "TaskFlow." Management is really optimistic about it, believing it will significantly streamline our workflow, particularly with remote teams. The idea is that it will integrate all the tools we currently rely on, like Slack, Trello, and Google Drive, into a single, cohesive platform.

Person B: That does sound intriguing! However, I have some reservations about the learning curve. Do you know if it’s going to be user-friendly?

Person A: From what I’ve seen so far, it looks quite intuitive. They’re also planning to hold several training sessions to ensure everyone gets up to speed quickly. The first session is scheduled for next Monday, so that should help ease any concerns.

Person B: Okay, that definitely puts my mind at ease. I suppose I’ll need to make it a point to attend that session. How does this new tool compare to what we’re currently using?

Person A: It’s reportedly much more efficient. We’ll be able to track project progress with much greater ease and receive real-time updates. Additionally, it features built-in analytics to assist us in monitoring performance and making informed decisions.

Person B: That certainly sounds promising! I just hope it doesn’t launch with a lot of bugs and issues.

Person A: Yeah, that’s always a concern when it comes to new software. However, they’ve been testing it thoroughly for a while now, so here’s hoping that everything goes smoothly at launch.

Person B: Fingers crossed! Let’s hope for the best. Thanks for sharing all this information with me!

Person A: No problem at all! I’m glad to help. I’ll see you at the training session
"""
))

In [ ]:
# Publish the model
from google.colab import userdata

repoNamr = "Taskil_SummaryBot"
hfToken = userdata.get("Taskil_token")

# save model and tokenizer
model.save_pretrained(repoName)
tokenizer.save_pretrained(repoName)

# push model and tokenizer to huggingface
model.push_to_hub(repoName, token=hfToken)
tokenizer.push_to_hub(repoName, token=hfToken)

In [ ]:
# Import the required modulesrch.cuda.is_available() else -1

# Load th
from transformers import pipeline

model = pipeline("summarization", model="TaskilMahmud/Taskil_SummaryBot", device=device)

# Example of zero-shot prompt for summarization
prompt = "Summarize the following text: " + sample_text
response = model(prompt)

print("Zero-shot Summary:", response[0]['summary_text'])